
# 説明

 * 「Optiver - Trading at the Close」コンペでは、"stock_id", "date_id", "seconds_in_bucket"のグループ化をして特徴量を生成した。
 * 特徴量の中には、複数回グループ化して関数を適用する処理をするものが存在。こうした処理を効率的に行うためにGroupbyChainOperatorクラスを実装
   

In [30]:
import seaborn as sns
import pandas as pd
import numpy as np
from numba import jit 

import sys
import os

from gbychain.code.groupby_chain_operator import GroupbyChainOperator


# GroupbyChainOperatorクラス

  * groupby関数を複数回適用する場合の処理を表現。
  * ここでは、初期値のinit_dfにgroupby関数を適用した回数をlevelと表し、初期値はlevel=0の扱いとする。
  * 各levelごとに生成した結果を保存。関数はfunc_dictに格納されている。列名は各levelの情報を数珠つなぎで2通りの命名法を用意
    1. 関数名を使用 
        * [init_dfの列名]\_[level=1で使用した関数名]\_[level=2で使用した関数名]_,,,,
            * 例: ask_price_mean_max 
    2. 番号を使用 
        * [init_dfの列名]\_[level=1で使用した関数のfunc_dictでの位置]\_[level=2で使用した関数のfunc_dictでの位置]_,,,,
            * 例: ask_price_1_0

## 初期化

## update_by_xxx関数

 * update_by_xxxとついた名前の関数でグループ化した処理を行う。関数名と説明は以下の通り




<div align="center">

| 関数名                 | 説明                                     |
|---------------------|----------------------------------------|
| update_by_agg       | agg関数を適用                               | 
| update_by_transform | transform関数を適用                         |
| update_by_apply2_agg | 2つの列x, yについてapply関数を適用して、1次元の出力結果を生成。、 |

</div>




 * この関数の引数について、以下に説明
    * groups(List[str]) - [groupby関数](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html)のbyに対応
    * indeces(List[str]|None) - transform関数でgroupsで指定した列に対応する順序を表す列。指定しなくても問題なし
    * tgt_names(List[str]|None) - groupby関数で関数に適用される列(ex:df.groupby(["a"])["b", "c"]の["b", "c"]を指す)、指定しない場合は前のlevelの生成結果の列名すべてが対象となる
    * func_dict(Dict[str, Any]|None) - 
        * groupby関数でagg, transform, applyで使用する関数のdict。前のlevelと同じものであれば省略は可能。
        - update_by_agg、update_by_transformが対象
    - col2_func_tpl_list(List[Tuple[str, str, Callable]])
        - groupby関数でapply、aggを順に適用。グループごとにCallable(str, str)の出力結果(1次元)が取得  
        - update_by_apply2_aggが対象
    * funcs_name(str|None) - func_dictを示す名前で、get_log関数に関連。
    * output_name(str|None) - Noneでない場合、get_log関数で各levelの結果の名前がoutput_nameになる。
   
 * アウトプットとしてGroupbyResultを生成。この生成結果はself.level_result_dictにlevelごと保存。 
    * GroupbyResultのoutput_dfがグループ化して関数を適用したdfで、groups, indeces, tgt_namesにfunc_dictをグループごとに適用した列で構成


 ## get_output_df関数

 * levelを指定してGroupbyResultのoutput_dfを取得。列名は関数名、番号を使用した記法の2つに対応
     
     * use_func_order(bool) - Trueの場合は番号を使用した記法、Falseの場合は関数名を使用した記法で列名を表示

## get_log関数

* 各levelで生成したdfの名前の一覧を取得
    * level=0のときは[init_dfの列名]_0
        * 例: init_df_0
    * level>=1のときは[手法]_[groupsの名前]_[関数dictの名前]_[level]
        * [手法]はupdate_by_xxx関数のxxxの値となる。
        * [groupsの名前]はupdate_by_xxx関数の引数のgroupsの値を用いて生成。具体的な生成法は以下の通り
            * groupsの値を"_"区切りで分割し、分割した要素の先頭をつなげる
            * つなげた上記の値を"-"で結合
                * 例: groups=["stock_id", "date_id"]の場合はsi-di
        * [関数dictの名前]はupdate_by_xxx関数の引数のfuncs_nameを使用。funcs_nameを指定しない場合はスキップ



## merge_all_level_df

 * すべてのlevelのGroupbyResultのoutput_dfを結合
    * level(int) - 0の場合は初期化で入れたinit_dfを出力。
     * use_func_order(bool) - Trueの場合は番号を使用した記法、Falseの場合は関数名を使用した記法で列名を表示

# 実行例



  * 「Optiver - Trading at the Close」コンペのデータを使用。

## Transform->Aggregate

 * GroupbyChainOperatorクラスの使用例を紹介する。適用するフローの例は以下の通り
    * 1."stock_id", "date_id"をグループ化して、移動平均を計算。(Transform)
    * 2."stock_id", "date_id"をグループ化して、1で計算した最大値を計算。(agg)
   

In [31]:
sub_train = pd.read_csv('../data/sub_train.csv')

### 実行

In [32]:
tgt_columns =  ["reference_price", "far_price"]

@jit(nopython=True)
def ewm_array(arr, span):
    n = len(arr)
    ewma = np.empty(n, dtype=np.float64)
    alpha = 2 / (span + 1)
    ewma[0] = arr[0]
    for i in range(1, n):
        ewma[i] = alpha * arr[i] + (1 - alpha) * ewma[i - 1]
    return ewma

func_dict = {    
    "ewm1":lambda x: x.values,
    "ewm5":lambda x: ewm_array(x.values, 5),
}
func_dict2 = {"mean":"mean", "zero": lambda x : 0, "std":"std"}

gcop = GroupbyChainOperator(sub_train, "sub_train")
(
    gcop.update_by_transform(["date_id", "stock_id"], ["seconds_in_bucket"], tgt_columns, func_dict)
     .update_by_agg( ["date_id", "stock_id"], ["seconds_in_bucket"], func_dict= func_dict2, funcs_name="max|zero|std", is_debug=False)
)



### get_output_df

 * 2回グループ化した結果は以下の通り

In [33]:
gcop.get_output_df(2, use_func_order=False)

,date_id,stock_id,reference_price_ewm1_mean,reference_price_ewm1_zero,reference_price_ewm1_std,reference_price_ewm5_mean,reference_price_ewm5_zero,reference_price_ewm5_std,far_price_ewm1_mean,far_price_ewm1_zero,far_price_ewm1_std,far_price_ewm5_mean,far_price_ewm5_zero,far_price_ewm5_std
0,0,0,0.999664,0,0.000435,0.999676,0,0.000381,0.999692,0,0.000521,NaN,0,NaN
1,0,1,0.999645,0,0.000659,0.999626,0,0.000548,0.994105,0,0.004752,NaN,0,NaN
2,0,2,0.999543,0,0.000382,0.999528,0,0.000260,0.999773,0,0.000321,NaN,0,NaN
3,1,0,0.999050,0,0.000391,0.999063,0,0.000324,1.009668,0,0.006552,NaN,0,NaN
4,1,1,1.000239,0,0.000597,1.000183,0,0.000482,1.000608,0,0.000418,NaN,0,NaN
5,1,2,1.000507,0,0.000995,1.000517,0,0.000892,1.028259,0,0.015385,NaN,0,NaN
6,2,0,0.998940,0,0.000498,0.998986,0,0.000478,0.998583,0,0.000233,NaN,0,NaN
7,2,1,0.996320,0,0.001538,0.996467,0,0.001635,0.966981,0,0.020634,NaN,0,NaN
8,2,2,0.999476,0,0.000632,0.999405,0,0.000517,0.998969,0,0.000333,NaN,0,NaN


 * 1回グループ化した結果は以下の通り

In [34]:
gcop.get_output_df(2, use_func_order=True)

,date_id,stock_id,reference_price_0_0,reference_price_0_1,reference_price_0_2,reference_price_1_0,reference_price_1_1,reference_price_1_2,far_price_0_0,far_price_0_1,far_price_0_2,far_price_1_0,far_price_1_1,far_price_1_2
0,0,0,0.999664,0,0.000435,0.999676,0,0.000381,0.999692,0,0.000521,NaN,0,NaN
1,0,1,0.999645,0,0.000659,0.999626,0,0.000548,0.994105,0,0.004752,NaN,0,NaN
2,0,2,0.999543,0,0.000382,0.999528,0,0.000260,0.999773,0,0.000321,NaN,0,NaN
3,1,0,0.999050,0,0.000391,0.999063,0,0.000324,1.009668,0,0.006552,NaN,0,NaN
4,1,1,1.000239,0,0.000597,1.000183,0,0.000482,1.000608,0,0.000418,NaN,0,NaN
5,1,2,1.000507,0,0.000995,1.000517,0,0.000892,1.028259,0,0.015385,NaN,0,NaN
6,2,0,0.998940,0,0.000498,0.998986,0,0.000478,0.998583,0,0.000233,NaN,0,NaN
7,2,1,0.996320,0,0.001538,0.996467,0,0.001635,0.966981,0,0.020634,NaN,0,NaN
8,2,2,0.999476,0,0.000632,0.999405,0,0.000517,0.998969,0,0.000333,NaN,0,NaN


 * use_func_order=True, Falseそれぞれで実行。両者の列名を比較

In [35]:
print("--------------use_func_order=True---------------")
print(gcop.get_output_df(2, use_func_order=True).columns)
print("--------------use_func_order=True---------------")
print(gcop.get_output_df(2, use_func_order=False).columns)

--------------use_func_order=True---------------
Index(['date_id', 'stock_id', 'reference_price_0_0', 'reference_price_0_1',
       'reference_price_0_2', 'reference_price_1_0', 'reference_price_1_1',
       'reference_price_1_2', 'far_price_0_0', 'far_price_0_1',
       'far_price_0_2', 'far_price_1_0', 'far_price_1_1', 'far_price_1_2'],
      dtype='object')
--------------use_func_order=True---------------
Index(['date_id', 'stock_id', 'reference_price_ewm1_mean',
       'reference_price_ewm1_zero', 'reference_price_ewm1_std',
       'reference_price_ewm5_mean', 'reference_price_ewm5_zero',
       'reference_price_ewm5_std', 'far_price_ewm1_mean',
       'far_price_ewm1_zero', 'far_price_ewm1_std', 'far_price_ewm5_mean',
       'far_price_ewm5_zero', 'far_price_ewm5_std'],
      dtype='object')


### get_log

  * グループごとのデータ生成の記録は以下の通り。

In [36]:
gcop.get_log()


['sub_train_0', 'transform_di-si_1', 'agg_di-si_max|zero|std_2']

### merge_all_level_df

 * use_func_order=True, Falseそれぞれで実行。両者の列名を比較

In [37]:
print("--------------use_func_order=True---------------")
print(gcop.merge_all_level_df(use_func_order=True).columns.tolist())
print("--------------use_func_order=False---------------")
print(gcop.merge_all_level_df(use_func_order=False).columns.tolist())

--------------use_func_order=True---------------
['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size', 'imbalance_buy_sell_flag', 'reference_price', 'matched_size', 'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price', 'ask_size', 'wap', 'target', 'time_id', 'row_id', 'reference_price_0', 'reference_price_1', 'far_price_0', 'far_price_1', 'reference_price_0_0', 'reference_price_0_1', 'reference_price_0_2', 'reference_price_1_0', 'reference_price_1_1', 'reference_price_1_2', 'far_price_0_0', 'far_price_0_1', 'far_price_0_2', 'far_price_1_0', 'far_price_1_1', 'far_price_1_2']
--------------use_func_order=False---------------
['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size', 'imbalance_buy_sell_flag', 'reference_price', 'matched_size', 'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price', 'ask_size', 'wap', 'target', 'time_id', 'row_id', 'reference_price_ewm1', 'reference_price_ewm5', 'far_price_ewm1', 'far_price_ewm5', 'reference_price_ewm1_mea

 * level=2での出力結果。use_func_order=Trueの場合は、各levelで適用した関数はfunc_dictのindexに対応
  *  use_func_order=Trueの場合、各levelで適用した関数はfunc_dictのindexに対応したものが

## apply2_agg->agg

* 適用するフローの例は以下の通り
    * 1."stock_id", "date_id"をグループ化して、reference_price < far_priceとなる数をカウント。(apply2_agg)
    * 2."date_id"をグループ化して、1で計算した合計を計算。(agg)b

### 実行

In [38]:
import itertools 
def temp(group, col1, col2) :
    return  (group[col1] < group[col2]).sum()

tgt_names_func_list = [("reference_price", "far_price", temp)] 

gcop = GroupbyChainOperator(sub_train, "sub_train")
(
    gcop.update_by_apply2_agg(["date_id", "stock_id"], ["seconds_in_bucket"], tgt_names_func_list)
     .update_by_agg(["date_id"], [""], func_dict = {"sum":"sum"})
)




### get_output_df

 * 2回グループ化した結果は以下の通り

In [39]:
gcop.get_output_df(2, use_func_order=False)

,date_id,reference_price-far_price_temp_sum
0,0,9
1,1,50
2,2,0


 * 1回グループ化した結果は以下の通り

In [40]:
gcop.get_output_df(2, use_func_order=True)

,date_id,reference_price-far_price_0_0
0,0,9
1,1,50
2,2,0


 * use_func_order=True, Falseそれぞれで実行。両者の列名を比較

In [41]:
print("--------------use_func_order=True---------------")
print(gcop.get_output_df(2, use_func_order=True).columns)
print("--------------use_func_order=True---------------")
print(gcop.get_output_df(2, use_func_order=False).columns)

--------------use_func_order=True---------------
Index(['date_id', 'reference_price-far_price_0_0'], dtype='object')
--------------use_func_order=True---------------
Index(['date_id', 'reference_price-far_price_temp_sum'], dtype='object')


### get_log

  * グループごとのデータ生成の記録は以下の通り。

In [42]:
gcop.get_log()


['sub_train_0', 'apply2-agg_di-si_1', 'agg_di_2']

### merge_all_level_df

 * use_func_order=True, Falseそれぞれで実行。両者の列名を比較

In [43]:
print("--------------use_func_order=True---------------")
print(gcop.merge_all_level_df(use_func_order=True).columns.tolist())
print("--------------use_func_order=False---------------")
print(gcop.merge_all_level_df(use_func_order=False).columns.tolist())

--------------use_func_order=True---------------
['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size', 'imbalance_buy_sell_flag', 'reference_price', 'matched_size', 'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price', 'ask_size', 'wap', 'target', 'time_id', 'row_id', 'reference_price-far_price_0', 'reference_price-far_price_0_0']
--------------use_func_order=False---------------
['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size', 'imbalance_buy_sell_flag', 'reference_price', 'matched_size', 'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price', 'ask_size', 'wap', 'target', 'time_id', 'row_id', 'reference_price-far_price_temp', 'reference_price-far_price_temp_sum']


## 他のデータについて

 * 以下はtitatnicのデータに対して適用した結果

In [44]:
titanic_df = sns.load_dataset("titanic")
groups = ["embarked", "sex"]
indices = ["age"]
tgt_columns= ["fare","sibsp"]

input_df = titanic_df.sort_values(indices)
groups2 = ["sex"]
indices2 = [""]


func_dict= {"max":"max", "min":"min"}

gcop = GroupbyChainOperator(input_df, "titanic")
(

    gcop.update_by_agg( groups, indices, tgt_columns, func_dict, funcs_name="max|min")
    .update_by_agg(groups2, indices2, output_name="test")
)

In [45]:
gcop.get_log()

['titanic_0', 'agg_e-s_max|min_1', 'test_2']

In [46]:
gcop.get_output_df(1, False)

,embarked,sex,fare_max,fare_min,sibsp_max,sibsp_min
0,C,female,512.3292,7.2250,2,0
1,C,male,512.3292,4.0125,2,0
2,Q,female,90.0000,6.7500,2,0
3,Q,male,90.0000,6.7500,4,0
4,S,female,263.0000,7.2500,8,0
5,S,male,263.0000,0.0000,8,0


In [47]:
gcop.get_output_df(2, False)

,sex,fare_max_max,fare_max_min,fare_min_max,fare_min_min,sibsp_max_max,sibsp_max_min,sibsp_min_max,sibsp_min_min
0,female,512.3292,90.0,7.25,6.75,8,2,0,0
1,male,512.3292,90.0,6.75,0.00,8,2,0,0


# テスト

In [48]:
import pandas as pd
df = pd.DataFrame(
    {
        "group1":["A"] * 6 + ["B"] * 4 ,
        "group2":["a"] * 3 + ["b"] * 4 + ["c"] * 3,
        "v1":[0,1,2,3,4,5,6,7,8,100],
        "v2":np.arange(20,30)[::-1]
        
    }
)
df

,group1,group2,v1,v2
0,A,a,0,29
1,A,a,1,28
2,A,a,2,27
3,A,b,3,26
4,A,b,4,25
5,A,b,5,24
6,B,b,6,23
7,B,c,7,22
8,B,c,8,21
9,B,c,100,20


## agg

In [49]:
func_dict = {"max":"max", "min":"min"}
func_dict2 = {"sum":"sum"}
gcop = GroupbyChainOperator(df)
(gcop.update_by_agg(["group1", "group2"],  tgt_names=["v1"], func_dict=func_dict)
 .update_by_agg(["group1"], func_dict=func_dict2))

In [50]:
gcop.get_output_df(0, False)

,group1,group2,v1,v2
0,A,a,0,29
1,A,a,1,28
2,A,a,2,27
3,A,b,3,26
4,A,b,4,25
5,A,b,5,24
6,B,b,6,23
7,B,c,7,22
8,B,c,8,21
9,B,c,100,20


In [51]:
gcop.get_output_df(1, False)

,group1,group2,v1_max,v1_min
0,A,a,2,0
1,A,b,5,3
2,B,b,6,6
3,B,c,100,7


In [53]:
gcop.merge_all_level_df(False)

,group1,group2,v1,v2,v1_max,v1_min,v1_max_sum,v1_min_sum
0,A,a,0,29,2,0,7,3
1,A,a,1,28,2,0,7,3
2,A,a,2,27,2,0,7,3
3,A,b,3,26,5,3,7,3
4,A,b,4,25,5,3,7,3
5,A,b,5,24,5,3,7,3
6,B,b,6,23,6,6,106,13
7,B,c,7,22,100,7,106,13
8,B,c,8,21,100,7,106,13
9,B,c,100,20,100,7,106,13


## apply2_agg

In [54]:
def lessthan(group, col1, col2) :
    return  (group[col1] > group[col2]).sum()

tgt_names_func_list = [("v1", "v2",  lessthan)] 

gcop = GroupbyChainOperator(df)
(
    gcop.update_by_apply2_agg(["group1", "group2"], "", tgt_names_func_list)
    .update_by_agg(["group1", "group2"], func_dict={"sum":"sum"})
     
)


,group1,group2,v1-v2_lessthan
0,A,a,0
1,A,b,0
2,B,b,0
3,B,c,1


In [55]:
gcop.get_output_df(0, False)

,group1,group2,v1,v2
0,A,a,0,29
1,A,a,1,28
2,A,a,2,27
3,A,b,3,26
4,A,b,4,25
5,A,b,5,24
6,B,b,6,23
7,B,c,7,22
8,B,c,8,21
9,B,c,100,20


In [56]:
gcop.get_output_df(1, False)

,group1,group2,v1-v2_lessthan
0,A,a,0
1,A,b,0
2,B,b,0
3,B,c,1


In [58]:
gcop.merge_all_level_df(False)

,group1,group2,v1,v2,v1-v2_lessthan,v1-v2_lessthan_sum
0,A,a,0,29,0,0
1,A,a,1,28,0,0
2,A,a,2,27,0,0
3,A,b,3,26,0,0
4,A,b,4,25,0,0
5,A,b,5,24,0,0
6,B,b,6,23,0,0
7,B,c,7,22,1,1
8,B,c,8,21,1,1
9,B,c,100,20,1,1


## transform

In [59]:
func_dict = {"cumsum":"cumsum"}
func_dict2 = {    
    "ewm1":lambda x: x.values,
    "ewm3":lambda x: ewm_array(x.values, 3),
}
# func_dict2 = {"sum":"sum"}
gcop = GroupbyChainOperator(df)
(gcop.update_by_transform(["group1", "group2"],  tgt_names=["v1", "v2"], func_dict=func_dict)
 .update_by_transform(["group1", "group2"], func_dict=func_dict2)
 )

In [60]:
gcop.get_output_df(0, False)

,group1,group2,v1,v2
0,A,a,0,29
1,A,a,1,28
2,A,a,2,27
3,A,b,3,26
4,A,b,4,25
5,A,b,5,24
6,B,b,6,23
7,B,c,7,22
8,B,c,8,21
9,B,c,100,20


In [61]:
gcop.get_output_df(1, False)

,group1,group2,v1_cumsum,v2_cumsum
0,A,a,0.0,29.0
1,A,a,1.0,57.0
2,A,a,3.0,84.0
3,A,b,3.0,26.0
4,A,b,7.0,51.0
5,A,b,12.0,75.0
6,B,b,6.0,23.0
7,B,c,7.0,22.0
8,B,c,15.0,43.0
9,B,c,115.0,63.0


In [63]:
gcop.merge_all_level_df(False)

,group1,group2,v1,v2,v1_cumsum,v2_cumsum,v1_cumsum_ewm1,v1_cumsum_ewm3,v2_cumsum_ewm1,v2_cumsum_ewm3
0,A,a,0,29,0.0,29.0,0.0,0.00,29.0,29.00
1,A,a,0,29,0.0,29.0,1.0,0.50,57.0,43.00
2,A,a,0,29,0.0,29.0,3.0,1.75,84.0,63.50
3,A,a,0,29,1.0,57.0,0.0,0.00,29.0,29.00
4,A,a,0,29,1.0,57.0,1.0,0.50,57.0,43.00
...,...,...,...,...,...,...,...,...,...,...
77,B,c,100,20,15.0,43.0,15.0,11.00,43.0,32.50
78,B,c,100,20,15.0,43.0,115.0,63.00,63.0,47.75
79,B,c,100,20,115.0,63.0,7.0,7.00,22.0,22.00
80,B,c,100,20,115.0,63.0,15.0,11.00,43.0,32.50


In [27]:
])

'a-b'